In [207]:
import pandas as pd 
import pickle 
from sklearn.metrics import confusion_matrix, precision_recall_curve, average_precision_score
from sklearn.metrics import auc, roc_auc_score, roc_curve, f1_score, accuracy_score, recall_score, precision_score, brier_score_loss
import numpy as np

In [215]:
test_datasets = [
    "test_df_global_to_save",
    'test_df_validation_1_to_save',
    'test_df_validation_2_to_save',
]
seed = 289

In [216]:
df_test_datasets = []
for i in range(len(test_datasets)):
    with open(f'{test_datasets[i]}_{seed}.pkl', 'rb') as pkl:
        f = pickle.load(pkl)
        all_labels = f['all_labels']
        all_final_predictions = f['all_final_predictions']
        all_pred_scores = f['all_pred_scores']
        uncertain_indices = f['uncertain_indices']
        
        all_pred_finger = all_pred_scores[0]
        all_pred_speech = all_pred_scores[1]
        all_pred_smile = all_pred_scores[2]
        
        print(len(all_labels), len(all_final_predictions), len(uncertain_indices), len(all_pred_finger))

    test_data = pd.read_csv(f'../../data/{test_datasets[i]}.csv')
    test_data['true_label'] = all_labels
    test_data['pred_score_finger'] = all_pred_finger
    test_data['pred_score_speech'] = all_pred_speech
    test_data['pred_score_smile'] = all_pred_smile
    test_data['pred_score_fusion'] = all_final_predictions
    test_data['uncertain_indices'] = uncertain_indices
    test_data['test_split'] = test_datasets[i][8:-8]

    
    
    if test_datasets[i] == 'test_df_validation_1_to_save':
        test_data = test_data[~test_data['row_id'].isin(['vdbmsErHFueM305Ch97LzcQOvvt2#2024-05-31'])]
        
    if test_datasets[i] == 'test_df_validation_2_to_save':
        test_data = test_data[~test_data['row_id'].isin(['vdbmsErHFueM305Ch97LzcQOvvt2#2023-04-26', 'vdbmsErHFueM305Ch97LzcQOvvt2#2023-05-01'])]


    df_test_datasets.append(test_data)

test_data = pd.concat(df_test_datasets)
test_data


162 162 162 162
92 92 92 92
69 69 69 69


,row_id,id,label,true_label,pred_score_finger,pred_score_speech,pred_score_smile,pred_score_fusion,uncertain_indices,test_split
0,NIHNT179KNNF4#2022-03-24,NIHNT179KNNF4,1.0,1.0,0.613548,0.568245,0.885688,0.988402,False,global
1,NIHNT179KNNF4#2023-06-30,NIHNT179KNNF4,1.0,1.0,0.472465,0.779543,0.280119,0.665972,False,global
2,NIHNT823CHAC3#2022-05-20,NIHNT823CHAC3,0.0,0.0,0.671538,0.671377,0.905939,0.976564,False,global
3,NIHNT823CHAC3#2021-05-07,NIHNT823CHAC3,0.0,0.0,0.710134,0.303872,0.577415,0.540549,False,global
4,NIHNT823CHAC3#2021-11-01,NIHNT823CHAC3,0.0,0.0,0.217497,0.467424,0.863133,0.112835,False,global
...,...,...,...,...,...,...,...,...,...,...
62,xSKwE2YgQFaXtUTTjbOyFhx3oCm2#2024-04-12,xSKwE2YgQFaXtUTTjbOyFhx3oCm2,1.0,1.0,0.831928,0.483484,0.604220,0.889474,False,validation_2
63,xtsnpDyq8BYQUtRaZjSjSCTGVoe2#2024-12-02,xtsnpDyq8BYQUtRaZjSjSCTGVoe2,0.0,0.0,0.424025,0.074897,0.347749,0.273126,False,validation_2
64,yWXLyTJ9MwUskybFC1AqHA1yoN73#2024-12-02,yWXLyTJ9MwUskybFC1AqHA1yoN73,0.0,0.0,0.449658,0.151100,0.311693,0.302878,False,validation_2
65,yxycFfgyJjMYeRRaw9ENTsXeE4H2#2024-12-02,yxycFfgyJjMYeRRaw9ENTsXeE4H2,0.0,0.0,0.352672,0.127602,0.127513,0.023653,False,validation_2


In [222]:
def safe_divide(numerator, denominator):
    if denominator == 0:
        return 0
    else:
        return numerator / denominator

In [223]:
def compute_metrics(y_true, y_pred_scores, threshold = 0.5):
    labels = np.asarray(y_true).reshape(-1)
    pred_scores = np.asarray(y_pred_scores).reshape(-1)
    preds = (pred_scores >= threshold)


    metrics = {}
    metrics['accuracy'] = accuracy_score(labels, preds)
    metrics['average_precision'] = average_precision_score(labels, pred_scores)
    metrics['auroc'] = roc_auc_score(labels, pred_scores)
    metrics['f1_score'] = f1_score(labels, preds)
    
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    metrics["confusion_matrix"] = {"tn":tn, "fp":fp, "fn":fn, "tp":tp}
    metrics["weighted_accuracy"] = (safe_divide(tp, tp + fp) + safe_divide(tn, tn + fn)) / 2.0

    '''
    True positive rate or recall or sensitivity: probability of identifying a positive case 
    (often called the power of a test)
    '''
    metrics['TPR'] = metrics['recall'] = metrics['sensitivity'] = recall_score(labels, preds)
    
    '''
    False positive rate: probability of falsely identifying someone as positive, who is actually negative
    '''
    metrics['FPR'] = safe_divide(fp, fp+tn)
    
    '''
    Positive Predictive Value: probability that a patient with a positive test result 
    actually has the disease
    '''
    metrics['PPV'] = metrics['precision'] = precision_score(labels, preds)
    
    '''
    Negative predictive value: probability that a patient with a negative test result 
    actually does not have the disease
    '''
    metrics['NPV'] = safe_divide(tn, tn+fn)
    
    '''
    True negative rate or specificity: probability of a negative test result, 
    conditioned on the individual truly being negative
    '''
    metrics['TNR'] = metrics['specificity'] = safe_divide(tn,(tn+fp))

    
    return metrics

In [224]:
compute_metrics(test_data['true_label'], test_data['pred_score_fusion'], threshold = 0.5)

{'accuracy': 0.803125,
 'average_precision': 0.8682044700620333,
 'auroc': 0.8699507389162561,
 'f1_score': 0.8264462809917356,
 'confusion_matrix': {'tn': 107, 'fp': 38, 'fn': 25, 'tp': 150},
 'weighted_accuracy': 0.8042392005157962,
 'TPR': 0.8571428571428571,
 'recall': 0.8571428571428571,
 'sensitivity': 0.8571428571428571,
 'FPR': 0.2620689655172414,
 'PPV': 0.7978723404255319,
 'precision': 0.7978723404255319,
 'NPV': 0.8106060606060606,
 'TNR': 0.7379310344827587,
 'specificity': 0.7379310344827587}

In [225]:
test_data_certain = test_data[~test_data['uncertain_indices']]
test_data_certain
compute_metrics(test_data_certain['true_label'], test_data_certain['pred_score_fusion'], threshold = 0.5)

{'accuracy': 0.8083067092651757,
 'average_precision': 0.8703813965186898,
 'auroc': 0.8740436034553681,
 'f1_score': 0.8305084745762712,
 'confusion_matrix': {'tn': 106, 'fp': 37, 'fn': 23, 'tp': 147},
 'weighted_accuracy': 0.810309234917425,
 'TPR': 0.8647058823529412,
 'recall': 0.8647058823529412,
 'sensitivity': 0.8647058823529412,
 'FPR': 0.25874125874125875,
 'PPV': 0.7989130434782609,
 'precision': 0.7989130434782609,
 'NPV': 0.8217054263565892,
 'TNR': 0.7412587412587412,
 'specificity': 0.7412587412587412}

In [226]:
test_data_certain_global = test_data_certain[test_data_certain['test_split'] == 'global']
compute_metrics(test_data_certain_global['true_label'], test_data_certain_global['pred_score_fusion'], threshold = 0.5)

{'accuracy': 0.80625,
 'average_precision': 0.8757657899381531,
 'auroc': 0.8751417004048583,
 'f1_score': 0.841025641025641,
 'confusion_matrix': {'tn': 47, 'fp': 18, 'fn': 13, 'tp': 82},
 'weighted_accuracy': 0.8016666666666666,
 'TPR': 0.8631578947368421,
 'recall': 0.8631578947368421,
 'sensitivity': 0.8631578947368421,
 'FPR': 0.27692307692307694,
 'PPV': 0.82,
 'precision': 0.82,
 'NPV': 0.7833333333333333,
 'TNR': 0.7230769230769231,
 'specificity': 0.7230769230769231}

In [227]:
test_data_certain_val_1 = test_data_certain[test_data_certain['test_split'] == 'validation_1']
compute_metrics(test_data_certain_val_1['true_label'], test_data_certain_val_1['pred_score_fusion'], threshold = 0.5)

{'accuracy': 0.8021978021978022,
 'average_precision': 0.8760579460263827,
 'auroc': 0.8717201166180758,
 'f1_score': 0.8235294117647058,
 'confusion_matrix': {'tn': 31, 'fp': 11, 'fn': 7, 'tp': 42},
 'weighted_accuracy': 0.8041211519364448,
 'TPR': 0.8571428571428571,
 'recall': 0.8571428571428571,
 'sensitivity': 0.8571428571428571,
 'FPR': 0.2619047619047619,
 'PPV': 0.7924528301886793,
 'precision': 0.7924528301886793,
 'NPV': 0.8157894736842105,
 'TNR': 0.7380952380952381,
 'specificity': 0.7380952380952381}

In [228]:
test_data_certain_val_2 = test_data_certain[test_data_certain['test_split'] == 'validation_2']
compute_metrics(test_data_certain_val_2['true_label'], test_data_certain_val_2['pred_score_fusion'], threshold = 0.5)

{'accuracy': 0.8225806451612904,
 'average_precision': 0.803277236328533,
 'auroc': 0.8589743589743589,
 'f1_score': 0.8070175438596491,
 'confusion_matrix': {'tn': 28, 'fp': 8, 'fn': 3, 'tp': 23},
 'weighted_accuracy': 0.8225806451612903,
 'TPR': 0.8846153846153846,
 'recall': 0.8846153846153846,
 'sensitivity': 0.8846153846153846,
 'FPR': 0.2222222222222222,
 'PPV': 0.7419354838709677,
 'precision': 0.7419354838709677,
 'NPV': 0.9032258064516129,
 'TNR': 0.7777777777777778,
 'specificity': 0.7777777777777778}

In [232]:
metadata = pd.read_csv('../../data/metadata_updated.csv')
# if Protocol is ParkTest, then the Participant ID is the Participant ID[20:]
def get_participant_id(row):
    if row['Protocol'] == 'ParkTest':
        return row['Participant_ID'][20:]
    else:
        return row['Participant_ID']

metadata['Participant_ID'] = metadata.apply(get_participant_id, axis=1)
metadata[metadata['Protocol'] == 'ParkTest'].head()

,Protocol,Participant_ID,Task,gender,age,race,pd,Diagnosis,gender_normalized,age_normalized,age_numeric,age_processed,race_normalized,env
543,ParkTest,530Z49,ahhhh,female,76.0,white,yes,1,Female,60 - 79,76.0,76.0,White,Home-Global
544,ParkTest,004Z53,ahhhh,male,76.0,white,no,0,Male,60 - 79,76.0,76.0,White,Home-Global
545,ParkTest,948Z87,ahhhh,female,74.0,white,yes,1,Female,60 - 79,74.0,74.0,White,Home-Global
546,ParkTest,872Z33,ahhhh,female,73.0,white,yes,1,Female,60 - 79,73.0,73.0,White,Home-Global
547,ParkTest,280Z18,ahhhh,female,61.0,white,yes,1,Female,60 - 79,61.0,61.0,White,Home-Global


In [233]:
# Merge test_data with selected columns from metadata on participant ID
test_data_w_metadata = pd.merge(
    test_data,
    metadata[[
        'Participant_ID',
        'Protocol',
        'gender_normalized',
        'age_processed',
        'age_normalized',
        'race_normalized',
        'Diagnosis'
    ]],
    left_on='id',
    right_on='Participant_ID',
    how='left'  # Use 'left' to preserve all test_data rows even if metadata is missing
)

In [234]:
test_data_w_metadata

,row_id,id,label,true_label,pred_score_finger,pred_score_speech,pred_score_smile,pred_score_fusion,uncertain_indices,test_split,Participant_ID,Protocol,gender_normalized,age_processed,age_normalized,race_normalized,Diagnosis
0,NIHNT179KNNF4#2022-03-24,NIHNT179KNNF4,1.0,1.0,0.613548,0.568245,0.885688,0.988402,False,global,NIHNT179KNNF4,SuperPD,Female,70.0,60 - 79,White,1
1,NIHNT179KNNF4#2023-06-30,NIHNT179KNNF4,1.0,1.0,0.472465,0.779543,0.280119,0.665972,False,global,NIHNT179KNNF4,SuperPD,Female,70.0,60 - 79,White,1
2,NIHNT823CHAC3#2022-05-20,NIHNT823CHAC3,0.0,0.0,0.671538,0.671377,0.905939,0.976564,False,global,NIHNT823CHAC3,SuperPD,Female,62.0,60 - 79,Black or African American,0
3,NIHNT823CHAC3#2021-05-07,NIHNT823CHAC3,0.0,0.0,0.710134,0.303872,0.577415,0.540549,False,global,NIHNT823CHAC3,SuperPD,Female,62.0,60 - 79,Black or African American,0
4,NIHNT823CHAC3#2021-11-01,NIHNT823CHAC3,0.0,0.0,0.217497,0.467424,0.863133,0.112835,False,global,NIHNT823CHAC3,SuperPD,Female,62.0,60 - 79,Black or African American,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,xSKwE2YgQFaXtUTTjbOyFhx3oCm2#2024-04-12,xSKwE2YgQFaXtUTTjbOyFhx3oCm2,1.0,1.0,0.831928,0.483484,0.604220,0.889474,False,validation_2,xSKwE2YgQFaXtUTTjbOyFhx3oCm2,ValidationStudy,Male,72.0,60 - 79,White,1
316,xtsnpDyq8BYQUtRaZjSjSCTGVoe2#2024-12-02,xtsnpDyq8BYQUtRaZjSjSCTGVoe2,0.0,0.0,0.424025,0.074897,0.347749,0.273126,False,validation_2,xtsnpDyq8BYQUtRaZjSjSCTGVoe2,ValidationStudy,Male,51.0,40 - 59,White,0
317,yWXLyTJ9MwUskybFC1AqHA1yoN73#2024-12-02,yWXLyTJ9MwUskybFC1AqHA1yoN73,0.0,0.0,0.449658,0.151100,0.311693,0.302878,False,validation_2,yWXLyTJ9MwUskybFC1AqHA1yoN73,ValidationStudy,Male,61.0,60 - 79,Others,0
318,yxycFfgyJjMYeRRaw9ENTsXeE4H2#2024-12-02,yxycFfgyJjMYeRRaw9ENTsXeE4H2,0.0,0.0,0.352672,0.127602,0.127513,0.023653,False,validation_2,yxycFfgyJjMYeRRaw9ENTsXeE4H2,ValidationStudy,Female,59.0,40 - 59,White,0


In [235]:
test_data_w_metadata.to_csv('../../data/test_data_w_metadata.csv', index=False)

In [236]:
df = pd.read_csv('../../data/test_data_w_metadata.csv')
df.head()

,row_id,id,label,true_label,pred_score_finger,pred_score_speech,pred_score_smile,pred_score_fusion,uncertain_indices,test_split,Participant_ID,Protocol,gender_normalized,age_processed,age_normalized,race_normalized,Diagnosis
0,NIHNT179KNNF4#2022-03-24,NIHNT179KNNF4,1.0,1.0,0.613548,0.568245,0.885688,0.988402,False,global,NIHNT179KNNF4,SuperPD,Female,70.0,60 - 79,White,1
1,NIHNT179KNNF4#2023-06-30,NIHNT179KNNF4,1.0,1.0,0.472465,0.779543,0.280119,0.665972,False,global,NIHNT179KNNF4,SuperPD,Female,70.0,60 - 79,White,1
2,NIHNT823CHAC3#2022-05-20,NIHNT823CHAC3,0.0,0.0,0.671538,0.671377,0.905939,0.976564,False,global,NIHNT823CHAC3,SuperPD,Female,62.0,60 - 79,Black or African American,0
3,NIHNT823CHAC3#2021-05-07,NIHNT823CHAC3,0.0,0.0,0.710134,0.303872,0.577415,0.540549,False,global,NIHNT823CHAC3,SuperPD,Female,62.0,60 - 79,Black or African American,0
4,NIHNT823CHAC3#2021-11-01,NIHNT823CHAC3,0.0,0.0,0.217497,0.467424,0.863133,0.112835,False,global,NIHNT823CHAC3,SuperPD,Female,62.0,60 - 79,Black or African American,0


In [237]:
# split the row_id column using # and set the 2nd half as the date
df['date'] = df['row_id'].str.split('#').str[1]
# convert the date column to datetime
df['date'] = pd.to_datetime(df['date'])
df.head()

,row_id,id,label,true_label,pred_score_finger,pred_score_speech,pred_score_smile,pred_score_fusion,uncertain_indices,test_split,Participant_ID,Protocol,gender_normalized,age_processed,age_normalized,race_normalized,Diagnosis,date
0,NIHNT179KNNF4#2022-03-24,NIHNT179KNNF4,1.0,1.0,0.613548,0.568245,0.885688,0.988402,False,global,NIHNT179KNNF4,SuperPD,Female,70.0,60 - 79,White,1,2022-03-24
1,NIHNT179KNNF4#2023-06-30,NIHNT179KNNF4,1.0,1.0,0.472465,0.779543,0.280119,0.665972,False,global,NIHNT179KNNF4,SuperPD,Female,70.0,60 - 79,White,1,2023-06-30
2,NIHNT823CHAC3#2022-05-20,NIHNT823CHAC3,0.0,0.0,0.671538,0.671377,0.905939,0.976564,False,global,NIHNT823CHAC3,SuperPD,Female,62.0,60 - 79,Black or African American,0,2022-05-20
3,NIHNT823CHAC3#2021-05-07,NIHNT823CHAC3,0.0,0.0,0.710134,0.303872,0.577415,0.540549,False,global,NIHNT823CHAC3,SuperPD,Female,62.0,60 - 79,Black or African American,0,2021-05-07
4,NIHNT823CHAC3#2021-11-01,NIHNT823CHAC3,0.0,0.0,0.217497,0.467424,0.863133,0.112835,False,global,NIHNT823CHAC3,SuperPD,Female,62.0,60 - 79,Black or African American,0,2021-11-01


In [247]:
metadata_facial = pd.read_csv('../../data/metadata_facial.csv')
metadata_speech = pd.read_csv('../../data/metadata_speech.csv')
metadata_finger_tapping = pd.read_csv('../../data/metadata_finger_tapping.csv')
metadata_finger_tapping_left = pd.read_csv('../../data/metadata_finger_tapping_left.csv')
metadata_finger_tapping_right = pd.read_csv('../../data/metadata_finger_tapping_right.csv',)



In [ ]:
test_data_w_metadata = df.copy()

# Ensure date is formatted consistently to match the filename pattern
test_data_w_metadata['date_str'] = pd.to_datetime(test_data_w_metadata['date']).dt.strftime('%Y-%m-%d')

def find_filename_by_substring(row, metadata_df):
    id_str = str(row['id'])
    date_str = row['date_str']
    
    matched = metadata_df[
        metadata_df['Filename'].str.contains(id_str) & 
        metadata_df['Filename'].str.contains(date_str)
    ]
    
    return matched['Filename'].values[0] if not matched.empty else None

# Assign filenames from each metadata group using substring search
test_data_w_metadata['facial_filename'] = test_data_w_metadata.apply(lambda row: find_filename_by_substring(row, metadata_facial), axis=1)
test_data_w_metadata['speech_filename'] = test_data_w_metadata.apply(lambda row: find_filename_by_substring(row, metadata_speech), axis=1)
test_data_w_metadata['finger_tapping_filename'] = test_data_w_metadata.apply(lambda row: find_filename_by_substring(row, metadata_finger_tapping), axis=1)
test_data_w_metadata['finger_tapping_left_filename'] = test_data_w_metadata.apply(lambda row: find_filename_by_substring(row, metadata_finger_tapping_left), axis=1)
test_data_w_metadata['finger_tapping_right_filename'] = test_data_w_metadata.apply(lambda row: find_filename_by_substring(row, metadata_finger_tapping_right), axis=1)

# Drop helper column if not needed
test_data_w_metadata.drop(columns='date_str', inplace=True)


In [249]:
test_data_w_metadata.head()

,row_id,id,label,true_label,pred_score_finger,pred_score_speech,pred_score_smile,pred_score_fusion,uncertain_indices,test_split,...,age_processed,age_normalized,race_normalized,Diagnosis,date,facial_filename,speech_filename,finger_tapping_filename,finger_tapping_left_filename,finger_tapping_right_filename
0,NIHNT179KNNF4#2022-03-24,NIHNT179KNNF4,1.0,1.0,0.613548,0.568245,0.885688,0.988402,False,global,...,70.0,60 - 79,White,1,2022-03-24,2022-03-24T13%3A32%3A36.977Z_NIHNT179KNNF4_smi...,2022-03-24T13%3A32%3A36.977Z_NIHNT179KNNF4_qui...,None,2022-03-24T13%3A32%3A36.977Z_NIHNT179KNNF4_fin...,2022-03-24T13%3A32%3A36.977Z_NIHNT179KNNF4_fin...
1,NIHNT179KNNF4#2023-06-30,NIHNT179KNNF4,1.0,1.0,0.472465,0.779543,0.280119,0.665972,False,global,...,70.0,60 - 79,White,1,2023-06-30,2023-06-30T15%3A13%3A51.098Z_NIHNT179KNNF4_smi...,2023-06-30T15%3A13%3A51.098Z_NIHNT179KNNF4_qui...,None,2023-06-30T15%3A13%3A51.098Z_NIHNT179KNNF4_fin...,2023-06-30T15%3A13%3A51.098Z_NIHNT179KNNF4_fin...
2,NIHNT823CHAC3#2022-05-20,NIHNT823CHAC3,0.0,0.0,0.671538,0.671377,0.905939,0.976564,False,global,...,62.0,60 - 79,Black or African American,0,2022-05-20,2022-05-20T19%3A15%3A10.454Z_NIHNT823CHAC3_smi...,2022-05-20T19%3A15%3A10.454Z_NIHNT823CHAC3_qui...,None,2022-05-20T19%3A15%3A10.454Z_NIHNT823CHAC3_fin...,2022-05-20T19%3A15%3A10.454Z_NIHNT823CHAC3_fin...
3,NIHNT823CHAC3#2021-05-07,NIHNT823CHAC3,0.0,0.0,0.710134,0.303872,0.577415,0.540549,False,global,...,62.0,60 - 79,Black or African American,0,2021-05-07,NIHNT823CHAC3-smile-2021-05-07T21-05-27-387Z-.mp4,NIHNT823CHAC3-quick_brown_fox-2021-05-07T20-59...,NIHNT823CHAC3-finger_tapping-2021-05-07T21-00-...,None,None
4,NIHNT823CHAC3#2021-11-01,NIHNT823CHAC3,0.0,0.0,0.217497,0.467424,0.863133,0.112835,False,global,...,62.0,60 - 79,Black or African American,0,2021-11-01,NIHNT823CHAC3-smile-2021-11-01T19-07-54-512Z-.mp4,NIHNT823CHAC3-quick_brown_fox-2021-11-01T19-01...,NIHNT823CHAC3-finger_tapping-2021-11-01T19-01-...,None,None


In [250]:
# Ensure predictions are interpreted as binary by thresholding at 0.5
test_data_w_metadata['misclassified_ufnet'] = test_data_w_metadata['true_label'] != (test_data_w_metadata['pred_score_fusion'] >= 0.5).astype(int)
test_data_w_metadata['misclassified_facial'] = test_data_w_metadata['true_label'] != (test_data_w_metadata['pred_score_smile'] >= 0.5).astype(int)
test_data_w_metadata['misclassified_speech'] = test_data_w_metadata['true_label'] != (test_data_w_metadata['pred_score_speech'] >= 0.5).astype(int)
test_data_w_metadata['misclassified_finger'] = test_data_w_metadata['true_label'] != (test_data_w_metadata['pred_score_finger'] >= 0.5).astype(int)


In [251]:
labeling_neurologist = pd.read_csv('../../data/labeling_neurologists_PARK.csv')
labeling_neurologist.columns
                                   

Index(['row_id', 'Unnamed: 1', 'PARK labels', 'Ray Labels',
       'Ray's correctness', 'Ruth Lables', 'Ruth's correctness',
       'Jamie Lables', 'Jamie's correctness', 'updated_at', 'PARK Labels',
       'True Labels', 'Ray Label', 'Ruth Label', 'Jamie Label'],
      dtype='object')

In [252]:
test_data_w_metadata_neurologist = pd.merge(
    test_data_w_metadata,
    labeling_neurologist[['row_id', 'Ray Label', 'Ruth Label', 'Jamie Label']],
    on='row_id',
    how='left'
)
test_data_w_metadata_neurologist.head()

,row_id,id,label,true_label,pred_score_finger,pred_score_speech,pred_score_smile,pred_score_fusion,uncertain_indices,test_split,...,finger_tapping_filename,finger_tapping_left_filename,finger_tapping_right_filename,misclassified_ufnet,misclassified_facial,misclassified_speech,misclassified_finger,Ray Label,Ruth Label,Jamie Label
0,NIHNT179KNNF4#2022-03-24,NIHNT179KNNF4,1.0,1.0,0.613548,0.568245,0.885688,0.988402,False,global,...,None,2022-03-24T13%3A32%3A36.977Z_NIHNT179KNNF4_fin...,2022-03-24T13%3A32%3A36.977Z_NIHNT179KNNF4_fin...,False,False,False,False,1.0,0.0,0.0
1,NIHNT179KNNF4#2023-06-30,NIHNT179KNNF4,1.0,1.0,0.472465,0.779543,0.280119,0.665972,False,global,...,None,2023-06-30T15%3A13%3A51.098Z_NIHNT179KNNF4_fin...,2023-06-30T15%3A13%3A51.098Z_NIHNT179KNNF4_fin...,False,True,False,True,1.0,1.0,0.0
2,NIHNT823CHAC3#2022-05-20,NIHNT823CHAC3,0.0,0.0,0.671538,0.671377,0.905939,0.976564,False,global,...,None,2022-05-20T19%3A15%3A10.454Z_NIHNT823CHAC3_fin...,2022-05-20T19%3A15%3A10.454Z_NIHNT823CHAC3_fin...,True,True,True,True,0.0,0.0,0.0
3,NIHNT823CHAC3#2021-05-07,NIHNT823CHAC3,0.0,0.0,0.710134,0.303872,0.577415,0.540549,False,global,...,NIHNT823CHAC3-finger_tapping-2021-05-07T21-00-...,None,None,True,True,False,True,NaN,NaN,NaN
4,NIHNT823CHAC3#2021-11-01,NIHNT823CHAC3,0.0,0.0,0.217497,0.467424,0.863133,0.112835,False,global,...,NIHNT823CHAC3-finger_tapping-2021-11-01T19-01-...,None,None,False,True,False,False,NaN,NaN,NaN


In [253]:
test_data_w_metadata_neurologist = test_data_w_metadata_neurologist.rename(columns={
    'row_id': 'unique_row_id', 
    'id': 'participant_id',  
    'Protocol': 'protocol', 
    'gender_normalized': 'gender',
    'age_processed': 'age', 
    'age_normalized': 'age_group', 
    'race_normalized': 'race', 
    'facial_filename': 'filename_smile', 
    'speech_filename': 'filename_speech', 
    'finger_tapping_filename': 'filename_finger_tapping',
    'finger_tapping_left_filename': 'filename_finger_tapping_left', 
    'finger_tapping_right_filename': 'filename_finger_tapping_right',
    'misclassified_ufnet': 'misclassified_fusion', 
    'misclassified_facial': 'misclassified_smile', 
    'Ray Label': 'neurologist_label_ray', 
    'Ruth Label': 'neurologist_label_ruth', 
    'Jamie Label': 'neurologist_label_jamie',
    'uncertain_indices': 'uncertain_flag'
})

In [ ]:
columns_to_keep = ['unique_row_id', 'participant_id', 'date', 'protocol', 'test_split',
        'gender', 'age', 'age_group', 'race', 
        'filename_smile', 'filename_speech', 'filename_finger_tapping',
        'filename_finger_tapping_left', 'filename_finger_tapping_right',
        'pred_score_smile', 'pred_score_speech', 'pred_score_finger', 'pred_score_fusion', 'true_label', 
        'misclassified_smile', 'misclassified_speech', 'misclassified_finger', 'misclassified_fusion', 'uncertain_flag',
        'neurologist_label_ray', 'neurologist_label_ruth', 'neurologist_label_jamie']

test_data_big_csv = test_data_w_metadata_neurologist[columns_to_keep]
test_data_big_csv.to_csv('../../data/test_data_big.csv', index=False)

# Statistical Tests

In [257]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score
from sklearn.utils import resample

In [258]:
df_test_results = test_data_big_csv
df_test_results['pred_park'] = (df_test_results['pred_score_fusion'] >= 0.5).astype(int)
df_test_results.head()

/tmp/ipykernel_3685726/1750590825.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_results['pred_park'] = (df_test_results['pred_score_fusion'] >= 0.5).astype(int)


,unique_row_id,participant_id,date,protocol,test_split,gender,age,age_group,race,filename_smile,...,true_label,misclassified_smile,misclassified_speech,misclassified_finger,misclassified_fusion,uncertain_flag,neurologist_label_ray,neurologist_label_ruth,neurologist_label_jamie,pred_park
0,NIHNT179KNNF4#2022-03-24,NIHNT179KNNF4,2022-03-24,SuperPD,global,Female,70.0,60 - 79,White,2022-03-24T13%3A32%3A36.977Z_NIHNT179KNNF4_smi...,...,1.0,False,False,False,False,False,1.0,0.0,0.0,1
1,NIHNT179KNNF4#2023-06-30,NIHNT179KNNF4,2023-06-30,SuperPD,global,Female,70.0,60 - 79,White,2023-06-30T15%3A13%3A51.098Z_NIHNT179KNNF4_smi...,...,1.0,True,False,True,False,False,1.0,1.0,0.0,1
2,NIHNT823CHAC3#2022-05-20,NIHNT823CHAC3,2022-05-20,SuperPD,global,Female,62.0,60 - 79,Black or African American,2022-05-20T19%3A15%3A10.454Z_NIHNT823CHAC3_smi...,...,0.0,True,True,True,True,False,0.0,0.0,0.0,1
3,NIHNT823CHAC3#2021-05-07,NIHNT823CHAC3,2021-05-07,SuperPD,global,Female,62.0,60 - 79,Black or African American,NIHNT823CHAC3-smile-2021-05-07T21-05-27-387Z-.mp4,...,0.0,True,False,True,True,False,NaN,NaN,NaN,1
4,NIHNT823CHAC3#2021-11-01,NIHNT823CHAC3,2021-11-01,SuperPD,global,Female,62.0,60 - 79,Black or African American,NIHNT823CHAC3-smile-2021-11-01T19-07-54-512Z-.mp4,...,0.0,True,False,False,False,False,NaN,NaN,NaN,0


In [259]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.utils import resample

def calculate_metrics_bootstrapped(true_labels, pred_labels, n_bootstraps=100):
    rng = np.random.RandomState(seed=42)

    # Convert to numpy arrays
    true_labels = np.array(true_labels)
    pred_labels = np.array(pred_labels)

    # Lists to store bootstrap results
    boot_auroc, boot_acc, boot_ppv, boot_npv, boot_sens, boot_spec, boot_f1 = [], [], [], [], [], [], []

    boot_count = 0
    while boot_count < n_bootstraps:
        # Sample indices with replacement
        indices = rng.choice(len(true_labels), size=len(true_labels), replace=True)
        
        # Sample labels
        y_true = true_labels[indices]
        y_pred = pred_labels[indices]

        try:
            auroc = roc_auc_score(y_true, y_pred)
        except ValueError:
            continue  # Retry this bootstrap

        acc = accuracy_score(y_true, y_pred)

        try:
            tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
        except ValueError:
            continue  # Retry this bootstrap

        # Compute metrics safely
        ppv = tp / (tp + fp) if (tp + fp) > 0 else np.nan
        npv = tn / (tn + fn) if (tn + fn) > 0 else np.nan
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else np.nan
        specificity = tn / (tn + fp) if (tn + fp) > 0 else np.nan
        f1_score = 2 * (ppv * sensitivity) / (ppv + sensitivity) if (ppv + sensitivity) > 0 else np.nan

        if np.isnan([ppv, npv, sensitivity, specificity, f1_score]).any():
            continue  # Retry this bootstrap

        # Append to result lists
        boot_auroc.append(auroc)
        boot_acc.append(acc)
        boot_ppv.append(ppv)
        boot_npv.append(npv)
        boot_sens.append(sensitivity)
        boot_spec.append(specificity)
        boot_f1.append(f1_score)

        boot_count += 1  # only increment if the sample is valid

    # Aggregate results
    bootstrap_results = {
        'AUROC': boot_auroc,
        'Accuracy': boot_acc,
        'PPV': boot_ppv,
        'NPV': boot_npv,
        'Sensitivity': boot_sens,
        'Specificity': boot_spec,
        'F1 Score': boot_f1
    }

    # Summary with mean ± 95% CI
    summary = {}
    for metric, values in bootstrap_results.items():
        mean_val = np.nanmean(values)
        lower_ci = np.nanpercentile(values, 2.5)
        upper_ci = np.nanpercentile(values, 97.5)
        margin = (upper_ci - lower_ci) / 2
        summary[metric] = f"{round(mean_val * 100, 1)} ± {round(margin * 100, 1)}"

    summary_df = pd.DataFrame.from_dict(summary, orient='index', columns=['Mean ± 95% CI'])
    return summary_df, bootstrap_results


In [260]:
df_test_results['test_split'].value_counts()

test_split
global          162
validation_1     91
validation_2     67
Name: count, dtype: int64

In [261]:
df_global = df_test_results[df_test_results['test_split'] == 'global']

true_labels_global = np.asarray(df_global['true_label'])
pred_labels_global = np.asarray(df_global['pred_park'])   

summary_df_global, bootstrap_results_global = calculate_metrics_bootstrapped(true_labels_global, pred_labels_global)
summary_df_global


,Mean ± 95% CI
AUROC,79.2 ± 6.6
Accuracy,80.7 ± 5.9
PPV,81.8 ± 7.1
NPV,78.7 ± 9.0
Sensitivity,86.7 ± 5.7
Specificity,71.7 ± 12.7
F1 Score,84.1 ± 5.5


In [262]:
df_val_1 = df_test_results[df_test_results['test_split'] == 'validation_1']
true_labels_val_1 = np.asarray(df_val_1['true_label'])
pred_labels_val_1 = np.asarray(df_val_1['pred_park'])   

summary_df_val_1, bootstrap_results_val_1 = calculate_metrics_bootstrapped(true_labels_val_1, pred_labels_val_1)
summary_df_val_1

,Mean ± 95% CI
AUROC,79.6 ± 8.5
Accuracy,80.1 ± 8.2
PPV,80.1 ± 10.8
NPV,80.2 ± 12.0
Sensitivity,84.8 ± 10.7
Specificity,74.5 ± 14.3
F1 Score,82.2 ± 7.8


In [263]:
df_val_2 = df_test_results[df_test_results['test_split'] == 'validation_2']

true_labels_val_2 = np.asarray(df_val_2['true_label'])
pred_labels_val_2 = np.asarray(df_val_2['pred_park'])   

summary_df_val_2, bootstrap_results_val_2 = calculate_metrics_bootstrapped(true_labels_val_2, pred_labels_val_2)
summary_df_val_2

,Mean ± 95% CI
AUROC,81.4 ± 9.1
Accuracy,81.0 ± 8.6
PPV,75.9 ± 13.9
NPV,86.0 ± 12.6
Sensitivity,84.2 ± 13.5
Specificity,78.5 ± 13.5
F1 Score,79.6 ± 10.0


In [264]:
from scipy.stats import mannwhitneyu
# from statsmodels.stats.multitest import multipletests
import itertools

# Define the distributions (re-using group_a, group_b, group_c as stand-ins)
distribution_sets = [
    bootstrap_results_global, 
    bootstrap_results_val_1,
    bootstrap_results_val_2
]
labels = ['Balanced Test Set', 'Validation Study 1', 'Validation Study 2']
metric_name = 'PPV'

# Perform pairwise Mann-Whitney U tests
results = []

for (i, j) in itertools.combinations(range(len(distribution_sets)), 2):
    group1 = distribution_sets[i][metric_name]
    group2 = distribution_sets[j][metric_name]
    label1 = labels[i]
    label2 = labels[j]
    stat, p = mannwhitneyu(group1, group2, alternative='two-sided')
    results.append({
        'Comparison': f'{label1} vs {label2}',
        'U Statistic': stat,
        'p-value': p
    })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)
results_df

,Comparison,U Statistic,p-value
0,Balanced Test Set vs Validation Study 1,6017.0,1.299756e-02
1,Balanced Test Set vs Validation Study 2,7541.5,5.333058e-10
2,Validation Study 1 vs Validation Study 2,6684.5,3.866663e-05


In [265]:
from scipy.stats import mannwhitneyu

# Define the distributions (re-using group_a, group_b, group_c as stand-ins)
distribution_sets = [
    bootstrap_results_global, 
    bootstrap_results_val_1,
    bootstrap_results_val_2
]
labels = ['Balanced Test Set', 'Validation Study 1', 'Validation Study 2']
metric_name = 'Sensitivity'

# Perform pairwise Mann-Whitney U tests
results = []

for (i, j) in itertools.combinations(range(len(distribution_sets)), 2):
    group1 = distribution_sets[i][metric_name]
    group2 = distribution_sets[j][metric_name]
    label1 = labels[i]
    label2 = labels[j]
    stat, p = mannwhitneyu(group1, group2, alternative='two-sided')
    results.append({
        'Comparison': f'{label1} vs {label2}',
        'U Statistic': stat,
        'p-value': p
    })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)
results_df

,Comparison,U Statistic,p-value
0,Balanced Test Set vs Validation Study 1,6139.0,0.005398
1,Balanced Test Set vs Validation Study 2,6421.5,0.000516
2,Validation Study 1 vs Validation Study 2,5358.5,0.381538


In [266]:
df = df_test_results.copy()

In [267]:
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency

np.random.seed(42)

# Observed counts (correct, incorrect) per PD stage
data = np.array([
    [11, 2],   # Stage 1
    [43, 4],   # Stage 2
    [8, 3]     # Stage 3
])

# Compute observed chi-square statistic
observed_stat, _, _, _ = chi2_contingency(data, correction=False)

print(observed_stat)

# Flatten into array of 1s and 0s
flat = np.concatenate([[1]*c + [0]*i for c, i in data])
group_sizes = data.sum(axis=1)

# Monte Carlo simulation
n_sim = 10000
simulated_stats = []

for _ in range(n_sim):
    shuffled = np.random.permutation(flat)
    reshaped = []
    start = 0
    for size in group_sizes:
        group = shuffled[start:start+size]
        correct = (group == 1).sum()
        incorrect = size - correct
        reshaped.append([correct, incorrect])
        start += size
    reshaped = np.array(reshaped)
    stat, _, _, _ = chi2_contingency(reshaped, correction=False)
    simulated_stats.append(stat)

simulated_stats = np.array(simulated_stats)
p_mc = (simulated_stats >= observed_stat).mean()

observed_stat, p_mc


2.9401730733233826


(2.9401730733233826, 0.157)

In [268]:
summary = {'gender': [{'group': 'Female',
   'mean': 0.22,
   'se': 0.03,
   'ci_lower': 0.16,
   'ci_upper': 0.29,
   'n_total': 171,
   'n_wrong': 38},
  {'group': 'Male',
   'mean': 0.17,
   'se': 0.03,
   'ci_lower': 0.11,
   'ci_upper': 0.23,
   'n_total': 147,
   'n_wrong': 25},
  {'group': 'Unknown',
   'mean': 0.0,
   'se': 0.0,
   'ci_lower': 0.0,
   'ci_upper': 0.0,
   'n_total': 2,
   'n_wrong': 0},
  {'group': 'all',
   'mean': 0.2,
   'se': 0.02,
   'ci_lower': 0.15,
   'ci_upper': 0.24,
   'n_total': 320,
   'n_wrong': 63}],
 'age_group': [{'group': '60 - 79',
   'mean': 0.2,
   'se': 0.03,
   'ci_lower': 0.14,
   'ci_upper': 0.25,
   'n_total': 194,
   'n_wrong': 39},
  {'group': '40 - 59',
   'mean': 0.18,
   'se': 0.04,
   'ci_lower': 0.1,
   'ci_upper': 0.26,
   'n_total': 85,
   'n_wrong': 15},
  {'group': 'Not Mentioned',
   'mean': 0.0,
   'se': 0.0,
   'ci_lower': 0.0,
   'ci_upper': 0.0,
   'n_total': 3,
   'n_wrong': 0},
  {'group': '>= 80',
   'mean': 0.0,
   'se': 0.0,
   'ci_lower': 0.0,
   'ci_upper': 0.0,
   'n_total': 7,
   'n_wrong': 0},
  {'group': '20 - 39',
   'mean': 0.33,
   'se': 0.09,
   'ci_lower': 0.16,
   'ci_upper': 0.5,
   'n_total': 27,
   'n_wrong': 9},
  {'group': '< 20',
   'mean': 0.0,
   'se': 0.0,
   'ci_lower': 0.0,
   'ci_upper': 0.0,
   'n_total': 4,
   'n_wrong': 0},
  {'group': 'all',
   'mean': 0.2,
   'se': 0.02,
   'ci_lower': 0.15,
   'ci_upper': 0.24,
   'n_total': 320,
   'n_wrong': 63}],
 'race': [{'group': 'white',
   'mean': 0.18,
   'se': 0.03,
   'ci_lower': 0.13,
   'ci_upper': 0.23,
   'n_total': 226,
   'n_wrong': 40},
  {'group': 'Non-white',
   'mean': 0.24,
   'se': 0.06,
   'ci_lower': 0.12,
   'ci_upper': 0.35,
   'n_total': 59,
   'n_wrong': 14},
  {'group': 'Unknown',
   'mean': 0.26,
   'se': 0.07,
   'ci_lower': 0.12,
   'ci_upper': 0.4,
   'n_total': 35,
   'n_wrong': 9},
  {'group': 'all',
   'mean': 0.2,
   'se': 0.02,
   'ci_lower': 0.15,
   'ci_upper': 0.24,
   'n_total': 320,
   'n_wrong': 63}]}

In [269]:
from statsmodels.stats.proportion import proportions_ztest


# Extract gender data for male and female only
gender_data = summary['gender']
female = next(g for g in gender_data if g['group'] == 'Female')
male = next(g for g in gender_data if g['group'] == 'Male')

# Gather counts
counts = [female['n_wrong'], male['n_wrong']]
totals = [female['n_total'], male['n_total']]
props = [counts[i] / totals[i] for i in range(2)]

# Check if sample size conditions are met for z-test
conditions_met = all([
    totals[i] * props[i] >= 5 and totals[i] * (1 - props[i]) >= 5
    for i in range(2)
])

# Perform test if valid
if conditions_met:
    stat, pval = proportions_ztest(counts, totals)
    gender_result = {
        "test": "z-test for two proportions",
        "z_statistic": round(stat, 4),
        "p_value": round(pval, 4),
        "female_error_rate": round(props[0], 3),
        "male_error_rate": round(props[1], 3),
        "sample_size_conditions_met": True
    }
else:
    result = {
        "error": "Sample size requirements not met for z-test",
        "sample_size_conditions_met": False
    }

gender_result

{'test': 'z-test for two proportions',
 'z_statistic': 1.1634,
 'p_value': 0.2447,
 'female_error_rate': 0.222,
 'male_error_rate': 0.17,
 'sample_size_conditions_met': True}

In [270]:
# Extract race data for White and Non-White
race_data = summary['race']
white = next(r for r in race_data if r['group'].lower() == 'white')
non_white = next(r for r in race_data if r['group'].lower() == 'non-white')

# Gather counts
counts = [white['n_wrong'], non_white['n_wrong']]
totals = [white['n_total'], non_white['n_total']]
props = [counts[i] / totals[i] for i in range(2)]

# Check sample size conditions
conditions_met_race = all([
    totals[i] * props[i] >= 5 and totals[i] * (1 - props[i]) >= 5
    for i in range(2)
])

# Perform test if valid
if conditions_met_race:
    stat, pval = proportions_ztest(counts, totals)
    race_result = {
        "test": "z-test for two proportions",
        "z_statistic": round(stat, 4),
        "p_value": round(pval, 4),
        "white_error_rate": round(props[0], 3),
        "non_white_error_rate": round(props[1], 3),
        "sample_size_conditions_met": True
    }
else:
    race_result = {
        "error": "Sample size requirements not met for z-test",
        "sample_size_conditions_met": False
    }

race_result

{'test': 'z-test for two proportions',
 'z_statistic': -1.0524,
 'p_value': 0.2926,
 'white_error_rate': 0.177,
 'non_white_error_rate': 0.237,
 'sample_size_conditions_met': True}

In [271]:
# Extract age group data and filter valid ones
age_data = summary['age_group']
valid_groups = [g for g in age_data if g['group'] in ['20 - 39', '40 - 59', '60 - 79']]

# Create contingency table: [correct, incorrect] for each group
age_contingency = []
age_labels = []

for group in valid_groups:
    correct = group['n_total'] - group['n_wrong']
    incorrect = group['n_wrong']
    age_contingency.append([correct, incorrect])
    age_labels.append(group['group'])

# Run chi-square test of independence
chi2_stat, p_val, dof, expected = chi2_contingency(age_contingency)

# Create expected frequencies DataFrame and check for < 5
expected_df = pd.DataFrame(expected, columns=['Correct_exp', 'Incorrect_exp'], index=age_labels)
expected_check = (expected_df < 5).any(axis=1)
any_cell_under_5 = expected_check.any()

# Format result
age_result = {
    "test": "Chi-square test of independence",
    "chi2_statistic": round(chi2_stat, 4),
    "p_value": round(p_val, 4),
    "degrees_of_freedom": dof,
    "age_groups_compared": age_labels,
    "all_expected_freqs_>=5": not any_cell_under_5
}

age_result


{'test': 'Chi-square test of independence',
 'chi2_statistic': 3.1602,
 'p_value': 0.206,
 'degrees_of_freedom': 2,
 'age_groups_compared': ['60 - 79', '40 - 59', '20 - 39'],
 'all_expected_freqs_>=5': True}

In [272]:
from statsmodels.stats.multitest import multipletests

# Raw p-values from your tests
pvals = [gender_result['p_value'], race_result['p_value'], age_result['p_value']]
labels = ['Gender', 'Race', 'Age Group']

# Apply FDR correction (Benjamini-Hochberg)
reject, pvals_corrected, _, _ = multipletests(pvals, alpha=0.05, method='fdr_bh')

# Display results
for i in range(len(pvals)):
    print(f"{labels[i]}: raw p = {pvals[i]:.4f}, FDR-corrected p = {pvals_corrected[i]:.4f}, significant = {reject[i]}")


Gender: raw p = 0.2447, FDR-corrected p = 0.2926, significant = False
Race: raw p = 0.2926, FDR-corrected p = 0.2926, significant = False
Age Group: raw p = 0.2060, FDR-corrected p = 0.2926, significant = False


In [276]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests


df['race_'] = df['race'].replace({
    'White': 'white',
    'Black or African American': 'Non-white',
    'American Indian or Alaska Native': 'Non-white',
    'Asian': 'Non-white',
    'Others': 'Non-white',
    'Not Mentioned': 'Unknown'
})

# Convert misclassified column to binary (1 = misclassified, 0 = correct)
df['error'] = df['misclassified_fusion'].astype(int)

# Filter rows to include only the desired levels
df_filtered = df[
    df['gender'].isin(['Female', 'Male']) &
    df['race_'].isin(['white', 'Non-white']) &
    df['age_group'].isin(['20 - 39', '40 - 59', '60 - 79'])
].copy()

# One-hot encode categorical predictors (drop_first avoids multicollinearity)
X = pd.get_dummies(df_filtered[['gender', 'race_', 'age_group']], drop_first=True)
X = sm.add_constant(X)
# Ensure all X columns are numeric
X = X.astype(float)
y = df_filtered['error']

# Fit logistic regression model
model = sm.Logit(y, X).fit()
pvals = model.pvalues

# Apply Benjamini-Hochberg FDR correction
reject, pvals_corrected, _, _ = multipletests(pvals, method='fdr_bh')

# Display results in a table
results = pd.DataFrame({
    'Variable': X.columns,
    'Coefficient': model.params.round(4),
    'Raw p-value': pvals.round(4),
    'FDR-adjusted p': pvals_corrected.round(4),
    'Significant (FDR < 0.05)': reject
})

results


Optimization terminated successfully.
         Current function value: 0.487766
         Iterations 6


,Variable,Coefficient,Raw p-value,FDR-adjusted p,Significant (FDR < 0.05)
const,const,-0.2303,0.6483,0.6483,False
gender_Male,gender_Male,-0.2460,0.4356,0.5445,False
race__white,race__white,-0.3206,0.3821,0.5445,False
age_group_40 - 59,age_group_40 - 59,-1.0503,0.0456,0.1905,False
age_group_60 - 79,age_group_60 - 79,-0.8444,0.0762,0.1905,False


In [277]:
from statsmodels.stats.contingency_tables import cochrans_q

df_clinician_ratings = df[~df['neurologist_label_ray'].isna()]

In [278]:
ray_preds = np.asarray(df_clinician_ratings['neurologist_label_ray'])                                     
ruth_preds = np.asarray(df_clinician_ratings['neurologist_label_ruth'])   
jamie_preds = np.asarray(df_clinician_ratings['neurologist_label_jamie'])   

group_prediction = (ray_preds + ruth_preds + jamie_preds >= 2).astype(int)

# Then compare boots_park['PPV'] to boots_group['PPV']


df_clinician_ratings['pred_park'] = (df_clinician_ratings['pred_score_fusion'] >= 0.5).astype(int)

park_preds = np.asarray(df_clinician_ratings['pred_park'])   

true_labels = np.asarray(df_clinician_ratings['true_label']) 

summary_ray, boots_ray = calculate_metrics_bootstrapped(true_labels, ray_preds)
summary_ruth, boots_ruth = calculate_metrics_bootstrapped(true_labels, ruth_preds)
summary_jamie, boots_jamie = calculate_metrics_bootstrapped(true_labels, jamie_preds)
summary_park, boots_park = calculate_metrics_bootstrapped(true_labels, park_preds)
summary_group, boots_group = calculate_metrics_bootstrapped(true_labels, group_prediction)

/tmp/ipykernel_3685726/2558307256.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clinician_ratings['pred_park'] = (df_clinician_ratings['pred_score_fusion'] >= 0.5).astype(int)


In [279]:
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests

def compare_park_to_clinicians(boots_park, boots_clinicians: dict, metrics: list):
    """
    Compare PARK's metrics to clinicians using one-sided Mann-Whitney U test (PARK < Clinician).
    
    Args:
        boots_park: dict of bootstrapped metrics for PARK
        boots_clinicians: dict of dicts (e.g., {'Ray': boots_ray, ...})
        metrics: list of metric names (e.g., ['PPV', 'Sensitivity'])
    
    Returns:
        A dictionary of test results and a printed summary message.
    """
    results = {}
    significantly_lower_metrics = []

    for metric in metrics:
        pvals = []
        pairs = []
        
        # inside the for metric loop
        for name, boots in boots_clinicians.items():
            dist_park = np.array(boots_park[metric])
            dist_clinician = np.array(boots[metric])

            # Check for valid data
            if np.all(np.isnan(dist_park)) or np.all(np.isnan(dist_clinician)):
                pvals.append(np.nan)
            else:
                try:
                    stat = mannwhitneyu(dist_park, dist_clinician, alternative='less')
                    pvals.append(stat.pvalue)
                except ValueError:
                    pvals.append(np.nan)
            
            pairs.append(f"PARK vs {name}")
        
        # FDR correction
        if all(np.isfinite(pvals)):
            reject, pvals_fdr, _, _ = multipletests(pvals, method='fdr_bh')
        else:
            reject = [False] * len(pvals)
            pvals_fdr = [np.nan] * len(pvals)
        
        # Store result
        results[metric] = {
            'pairs': pairs,
            'raw_pvals': pvals,
            'fdr_pvals': pvals_fdr,
            'reject': reject
        }

        # Track significance
        if any(reject):
            significantly_lower_metrics.append(metric)
        
        # Print detailed result per metric
        print(f"\nMetric: {metric}")
        for i, pair in enumerate(pairs):
            print(f"{pair}: raw p = {pvals[i]:.4f}, FDR-adjusted p = {pvals_fdr[i]:.4f}, significant = {reject[i]}")
    
    # Summary message
    if significantly_lower_metrics:
        print(f"\n⚠️ PARK showed significantly lower performance (FDR < 0.05) than at least one clinician for: {', '.join(significantly_lower_metrics)}")
    else:
        print("\n✅ PARK did not show significantly lower performance than any clinician for any metric (FDR-adjusted p > 0.05).")
    
    return results


In [280]:
# Define the clinicians and the metrics to compare
clinicians_boots = {
    'Ray': boots_ray,
    'Ruth': boots_ruth,
    'Jamie': boots_jamie,
    'Group': boots_group
}
metrics_to_compare = ['Accuracy', 'Specificity', 'Sensitivity', 'PPV', 'NPV', 'F1 Score']

# Run the function
results = compare_park_to_clinicians(boots_park, clinicians_boots, metrics_to_compare)



Metric: Accuracy
PARK vs Ray: raw p = 0.9999, FDR-adjusted p = 1.0000, significant = False
PARK vs Ruth: raw p = 0.0001, FDR-adjusted p = 0.0003, significant = True
PARK vs Jamie: raw p = 1.0000, FDR-adjusted p = 1.0000, significant = False
PARK vs Group: raw p = 0.4405, FDR-adjusted p = 0.8810, significant = False

Metric: Specificity
PARK vs Ray: raw p = 1.0000, FDR-adjusted p = 1.0000, significant = False
PARK vs Ruth: raw p = 1.0000, FDR-adjusted p = 1.0000, significant = False
PARK vs Jamie: raw p = 0.3216, FDR-adjusted p = 1.0000, significant = False
PARK vs Group: raw p = 1.0000, FDR-adjusted p = 1.0000, significant = False

Metric: Sensitivity
PARK vs Ray: raw p = 0.0000, FDR-adjusted p = 0.0000, significant = True
PARK vs Ruth: raw p = 0.0000, FDR-adjusted p = 0.0000, significant = True
PARK vs Jamie: raw p = 1.0000, FDR-adjusted p = 1.0000, significant = False
PARK vs Group: raw p = 0.0002, FDR-adjusted p = 0.0002, significant = True

Metric: PPV
PARK vs Ray: raw p = 1.0000,